In [1]:
# 提取单次出行轨迹
# 自适应调整时间阈值：根据不同时刻、不同路段调整单次出行划分的时间阈值
# 2025/01/19 by chenxinyi

import networkx as nx
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 读取数据预处理之后的数据
df = pd.read_csv(r'D:\24216373郑金涛中微观研究\center_valid_df.csv', encoding='gbk')
df['GCSJ'] = pd.to_datetime(df["GCSJ"])     # 时间格式转换
df['KKBH'] = df["KKBH"].astype("str")       # 卡口编号格式转换
df.head(3)

In [3]:
# 读取路网拓扑结构
G = nx.read_gml(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\1.构建路网拓扑结构\center_roadnet')
pos = np.load(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\1.构建路网拓扑结构\center_pos.npy', allow_pickle=True).item()
pos_name = list(pos.keys())
pos_coord = list(pos.values())
realpos = np.load(r'C:\01 毕业论文\7.论文代码和结果\硕士毕业论文代码\应用案例分析\车辆路径重构\1.构建路网拓扑结构\center_realpos.npy')
realpos_name = list(realpos[0])
bayonet_name = list(realpos[1])

In [4]:
# 选取固定行程时间阈值
TravelTime = pd.read_csv(r'D:\24216373郑金涛中微观研究\车辆轨迹划分\构建自适应行程时间阈值/全样本相邻卡口记录时间间隔.csv')
B = TravelTime['time'].quantile(q=0.70)   # 选取70%~80%位路网行程时间作为固定时间阈值
print("固定时间阈值：B = {}s".format(B))

In [5]:
# 读取每一相邻卡口之间的动态行程时间阈值
kkod_time = np.load(r'D:\24216373郑金涛中微观研究\车辆轨迹划分\构建自适应行程时间阈值\kkod_time_threshold.npy')
kkod_time_df = pd.DataFrame(kkod_time, columns=['kkod', 'period', 'min', 'mean', 'max'])
timesplit = [[7,8,17,18],
             [9,10,11,12,13,14,15,16,19,20,21],
             [0,1,2,3,4,5,6,22,23]]

# 读取卡口数据中的所有出行车辆
VehicleList = df['CLTMBH'].drop_duplicates()
VehicleList = list(VehicleList)
print("出行车辆数:", len(VehicleList))

In [7]:
from tqdm import tqdm
import numpy as np

save_path = r'D:\24216373郑金涛中微观研究\中心城区单次出行轨迹结果'
ind = 0
path_pos = []      # 存储单次出行轨迹经过的路网节点
path_bayonet = []  # 存储单次出行轨迹经过的卡口设备
path_time = []     # 存储经过每一卡口设备的时间
path_start_time = []    # 存储起始时间
path_stop_time = []     # 存储结束时间
path_travel = []        # 存储行程时间
path_cltmbh = []        # 存储车辆号牌

for i in tqdm(range(ind, len(VehicleList)), desc="处理车辆"):

    # todo 提取单车出行记录
    vt = df[df['CLTMBH'] == VehicleList[i]].reset_index(drop=True)

    # 如果仅有1条数据记录，则跳过
    if len(vt) <= 1:
        continue

    CLTMBH = vt.loc[0, 'CLTMBH']
    vt_pos = []      # 存储单车出行经过的路网节点
    vt_bayonet = []  # 存储单车出行经过的卡口设备
    vt_time = []     # 存储经过每一卡口设备的时间

    # todo 查找单车出行记录对应的路网节点
    for j in range(0, len(vt)):
        GCSJ = vt.loc[j, 'GCSJ']
        KKBH = vt.loc[j, 'KKBH']
        index = bayonet_name.index(KKBH)
        vt_pos.append(realpos_name[index])
        vt_bayonet.append(KKBH)
        vt_time.append(GCSJ)


    # todo 划分单次出行轨迹

    # 新建单次出行记录
    st_pos = []
    st_bayonet = []
    st_time = []

    # 遍历单车出行记录
    for z in range(1, len(vt_pos)):

        # todo 查询相邻卡口对应的合理行程时间区间

        kkod = str(vt_bayonet[z - 1]) + '-' + str(vt_bayonet[z])
        kkhour = vt_time[z].hour
        test_range = kkod_time_df[kkod_time_df['kkod'] == kkod]

        # 1. 如果未找到相邻卡口对应的合理行程时间区间，则取默认区间
        if test_range.empty: 
            mintime = 1
            maxtime = round(1.5 * B)

        # 2. 如果合理行程时间区间不区分高峰、低峰和平峰
        elif len(test_range) == 1:
            kkod_mintime = float(test_range.iloc[0, 2])    # 合理区间下限
            kkod_maxtime = float(test_range.iloc[0, 4])    # 合理区间上限
            kkod_interval = (kkod_maxtime - kkod_mintime)  # 合理区间上限和下限之差

            # 如果合理区间下限大于固定时间阈值且上下限之差小于60s/或者存在nan值，则取默认区间
            if test_range.iloc[0, 2] == 'nan':
                mintime = 1
                maxtime = round(1.5 * B)
            elif  kkod_mintime >= B and kkod_interval < 60:
                mintime = 1
                maxtime = round(1.5 * B)
            else:
                mintime = int(kkod_mintime)
                maxtime = round(kkod_maxtime)

        # 3. 如果合理行程时间区间区分高峰、低峰和平峰
        else:

            # 筛选不同交通时段的合理行程时间区间
            if kkhour in timesplit[0]:
                test_range = test_range[test_range['period'] == 'high']
            elif kkhour in timesplit[1]:
                test_range = test_range[test_range['period'] == 'flat']
            else:
                test_range = test_range[test_range['period'] == 'low']

            kkod_mintime = float(test_range.iloc[0, 2])    # 合理区间下限
            kkod_maxtime = float(test_range.iloc[0, 4])    # 合理区间上限
            kkod_interval = (kkod_maxtime - kkod_mintime)  # 合理区间上限和下限之差

            # 如果合理区间下限大于固定时间阈值且上下限之差小于60s/或者存在nan值，则取默认区间
            if test_range.iloc[0, 2] == 'nan':
                mintime = 1
                maxtime = round(1.5 * B)
            elif kkod_mintime >= B and kkod_interval < 60:
                mintime = 1
                maxtime = round(1.5 * B)
            else:
                mintime = int(kkod_mintime)
                maxtime = round(kkod_maxtime)

        # 合理放缩最大行程时间阈值
        maxtime = min(maxtime, round(1.5 * B))


        # todo 计算车辆在相邻卡口之间的实际行驶时间

        if st_time == []:
            last_time = vt_time[z - 1]
        else:
            last_time = st_time[-1]
        ti = (vt_time[z] - last_time).total_seconds()

        # 打印最终的合理行程时间区间
        # print(f"{vt_time[z]}  KKBH: {vt_bayonet[z]}")
        # print(f"实际行驶时间: {ti}  行程时间阈值: [{mintime}, {maxtime}]")


        # todo 根据合理行程时间区间，划分车辆单次出行

        # 1. 若车辆前后经过的卡口点位不一致
        if vt_bayonet[z] != vt_bayonet[z - 1]:

            # 1.1 若∆𝑡在合理行程时间区间，则b_(𝑗-1)和b_𝑗属于同一次单次出行
            if ti >= mintime and ti <= maxtime and ti != 0:  
                # 1.1.1 当遍历到单车出行信息中第二个卡口数据
                if z == 1:  
                    st_pos.extend((vt_pos[z - 1], vt_pos[z]))
                    st_bayonet.extend((vt_bayonet[z - 1], vt_bayonet[z]))
                    st_time.extend((vt_time[z - 1], vt_time[z]))
                # 1.1.2 当遍历到单车出行信息中最后一个卡口数据
                elif z == (len(vt_pos) - 1):  
                    st_pos.append(vt_pos[z])
                    st_bayonet.append(vt_bayonet[z])
                    st_time.append(vt_time[z])
                    if len(st_pos) > 1 and (st_time[-1] - st_time[0]).total_seconds() > 300: # 单次出行时间默认＞5min
                        path_pos.append(st_pos)
                        path_bayonet.append(st_bayonet)
                        path_time.append(st_time)
                        path_start_time.append(st_time[0])
                        path_stop_time.append(st_time[-1])
                        path_travel.append((st_time[-1] - st_time[0]).total_seconds())
                        path_cltmbh.append(CLTMBH)
                # 1.1.3 当遍历到单车出行信息中的中间卡口数据
                else:  
                    st_pos.append(vt_pos[z])
                    st_bayonet.append(vt_bayonet[z])
                    st_time.append(vt_time[z])

            # 1.2 若∆𝑡大于行程时间区间上限，则b_(𝑗-1)是当前出行的终点， b_𝑗是下一次出行的起点
            elif ti > maxtime:  
                if len(st_pos) > 1 and (st_time[-1] - st_time[0]).total_seconds() > 300:
                    path_pos.append(st_pos)
                    path_bayonet.append(st_bayonet)
                    path_time.append(st_time)
                    path_start_time.append(st_time[0])
                    path_stop_time.append(st_time[-1])
                    path_travel.append((st_time[-1] - st_time[0]).total_seconds())
                    path_cltmbh.append(CLTMBH)
                st_pos = [vt_pos[z]]
                st_bayonet = [vt_bayonet[z]]
                st_time = [vt_time[z]]

            # 1.3 若∆𝑡小于行程时间区间下限，则删除b_𝑗
            else:  
                continue
            # print(st_bayonet)

        # 2. 若车辆前后经过的卡口点位一致且ti>B，则b_(𝑗-1)是当前出行的终点， b_𝑗是下一次出行的起点
        elif ti > B:
            if len(st_pos) > 1 and (st_time[-1] - st_time[0]).total_seconds() > 300:
                path_pos.append(st_pos)
                path_bayonet.append(st_bayonet)
                path_time.append(st_time)
                path_start_time.append(st_time[0])
                path_stop_time.append(st_time[-1])
                path_travel.append((st_time[-1] - st_time[0]).total_seconds())
                path_cltmbh.append(CLTMBH)
            st_pos = [vt_pos[z]]
            st_bayonet = [vt_bayonet[z]]
            st_time = [vt_time[z]]

        # 3. 若车辆前后经过的卡口点位一致且ti≤B，则删除b_(𝑗+1)
        else:
            continue

    # print("车辆序号:", i)

    # 定期打印算法运行时间和自动保存
    if i in list(np.arange(5000, 420001, 5000)):
        print(f"第{i}辆车 轨迹划分完成")
        np.save(save_path + '/' + 'path_pos.npy', path_pos)
        np.save(save_path + '/' + 'path_bayonet.npy', path_bayonet)
        np.save(save_path + '/' + 'path_time.npy', path_time)
        np.save(save_path + '/' + 'path_start_time.npy', path_start_time)
        np.save(save_path + '/' + 'path_stop_time.npy', path_stop_time)
        np.save(save_path + '/' + 'path_travel.npy', path_travel)
        np.save(save_path + '/' + 'path_cltmbh.npy', path_cltmbh)

print(1)

print("出行轨迹数: ", len(path_pos))





In [ ]:
# 打印单次出行划分结果
print('出行划分后的研究车辆:', len(list(set(path_cltmbh))))
print('单次出行路径:', len(path_cltmbh), len(path_start_time), len(path_stop_time), len(path_travel),
      len(path_pos), len(path_bayonet), len(path_time))

idx = 390780
interval = 1
print('车辆号牌:', path_cltmbh[idx:idx+interval])
print('起始时间:', path_start_time[idx:idx+interval])
print('结束时间:', path_stop_time[idx:idx+interval])
print('总行程时间:', path_travel[idx:idx+interval])
print('途经路网节点:', path_pos[idx:idx+interval])
print('途经卡口设备:', path_bayonet[idx:idx+interval])
print('对应的过车时间:', path_time[idx:idx+interval])

In [ ]:
# 保存数据
np.save(save_path + '/' + 'path_pos.npy', path_pos)
np.save(save_path + '/' + 'path_bayonet.npy', path_bayonet)
np.save(save_path + '/' + 'path_time.npy', path_time)
np.save(save_path + '/' + 'path_start_time.npy', path_start_time)
np.save(save_path + '/' + 'path_stop_time.npy', path_stop_time)
np.save(save_path + '/' + 'path_travel.npy', path_travel)
np.save(save_path + '/' + 'path_cltmbh.npy', path_cltmbh)

In [ ]:
# 统计每次单次出行经过的卡口数
path_bayonet_number = []
for i in range(0, len(path_bayonet)):
    num = len(path_bayonet[i])
    path_bayonet_number.append(num)
    
print('单次出行平均卡口记录数:', round(np.mean(path_bayonet_number), 2))
print('中位数:', np.median(path_bayonet_number), '最小记录数:', min(path_bayonet_number), '最大记录数:', max(path_bayonet_number))

np.save(save_path + '/' + 'path_bayonet_number.npy', path_bayonet_number)

In [ ]:
# 读取已经处理好的数据

file = r'F:\陈心怡毕业文件\01 毕业论文\7.论文代码和结果\案例分析\数据\路径重构结果\中心城区单次出行轨迹结果'
path_pos = list(np.load(file + '/' + 'path_pos.npy', allow_pickle=True))      # 存储单次出行轨迹经过的路网节点
path_bayonet = list(np.load(file + '/' + 'path_bayonet.npy', allow_pickle=True))  # 存储单次出行轨迹经过的卡口设备
path_time = list(np.load(file + '/' + 'path_time.npy', allow_pickle=True))     # 存储经过每一卡口设备的时间
path_start_time = list(np.load(file + '/' + 'path_start_time.npy', allow_pickle=True))   # 存储起始时间
path_stop_time = list(np.load(file + '/' + 'path_stop_time.npy', allow_pickle=True))     # 存储结束时间
path_travel = list(np.load(file + '/' + 'path_travel.npy'))        # 存储行程时间
path_cltmbh = list(np.load(file + '/' + 'path_cltmbh.npy'))        # 存储车辆号牌

# print('已处理单次出行数:',len(path_pos), len(path_travel), len(path_bayonet), len(path_time), 
#                          len(path_start_time), len(path_stop_time), len(path_cltmbh))
# print('查看后3条记录:', path_pos[-3:])
# print('行程时间:', path_travel[-3:])
# print('车辆号牌:', path_cltmbh[-3:])
# print('\n最后一条记录的车辆号牌在VehicleList中的索引:', VehicleList.index(path_cltmbh[-1]))
# ind = VehicleList.index(path_cltmbh[-1]) + 1

In [ ]:
# 优化后的轨迹提取与划分 (Jupyter Notebook)


import pandas as pd
import numpy as np
import time
from joblib import Parallel, delayed
import warnings

warnings.filterwarnings('ignore')

# --- 保存路径 (原格式) ---
save_path = r'D:\24216373郑金涛中微观研究\中心城区单次出行轨迹结果'

# --- 一次性全局预处理 ---
# 1. 读取与预处理
df = pd.read_csv(r'D:\24216373郑金涛中微观研究\center_valid_df.csv', encoding='gbk', parse_dates=['GCSJ'])
# 构建卡口到真实节点映射
realpos_arr = np.load(r'D:\24216373郑金涛中微观研究\center_realpos.npy', allow_pickle=True)
realpos_name = realpos_arr[0]
bayonet_name = realpos_arr[1]
bayonet_to_pos = dict(zip(bayonet_name, realpos_name))

# 2. 固定时间阈值 B
TravelTime = pd.read_csv(r'D:\24216373郑金涛中微观研究\全样本相邻卡口记录时间间隔.csv')
B = TravelTime['time'].quantile(0.70)

# 3. 动态阈值映射
kkod_np = np.load(r'D:\24216373郑金涛中微观研究\kkod_time_threshold.npy', allow_pickle=True)
kkod_df = pd.DataFrame(kkod_np, columns=['kkod','period','min','mean','max'])
kkod_map = {}
for _, row in kkod_df.iterrows():
    kkod_map.setdefault(row['kkod'], {})[row['period']] = (float(row['min']), float(row['max']))

# 4. 时段划分
timesplit = {
    'high': set([7,8,17,18]),
    'flat': set(range(9,17)) | set([19,20,21]),
    'low': set(range(0,7)) | set([22,23])
}

# 5. 准备所有车辆列表
vehicle_list = df['CLTMBH'].unique()

# --- 核心函数: 单车轨迹分段 ---
def process_vehicle(cltmbh):
    vt = df[df['CLTMBH'] == cltmbh].sort_values('GCSJ')
    if len(vt) <= 1:
        return []
    vt['pos'] = vt['KKBH'].map(bayonet_to_pos)
    vt['kkod'] = vt['KKBH'].shift(1).fillna('') + '-' + vt['KKBH']
    vt['prev_time'] = vt['GCSJ'].shift(1)
    vt = vt.dropna(subset=['prev_time'])
    segments = []
    st_indices = [vt.index[0]]

    for idx, row in vt.iterrows():
        kk = row['kkod']
        hour = row['GCSJ'].hour
        period = next((p for p,hset in timesplit.items() if hour in hset), None)
        if kk not in kkod_map:
            mint, maxt = 1, round(1.5 * B)
        else:
            peri_map = kkod_map[kk]
            if 'all' in peri_map or period not in peri_map:
                mint, maxt = peri_map.get('all', (1, round(1.5 * B)))
            else:
                mint, maxt = peri_map[period]
        maxt = min(maxt, round(1.5 * B))
        ti = (row['GCSJ'] - row['prev_time']).total_seconds()
        same_bayonet = (vt.loc[idx,'KKBH'] == vt.loc[idx-1 if idx-1 in vt.index else idx,'KKBH'])

        if not same_bayonet and mint <= ti <= maxt:
            st_indices.append(idx)
        else:
            start, end = st_indices[0], st_indices[-1]
            duration = (vt.loc[end,'GCSJ'] - vt.loc[start,'GCSJ']).total_seconds()
            if len(st_indices) > 1 and duration > 300:
                segments.append({
                    'CLTMBH': cltmbh,
                    'pos': vt.loc[st_indices,'pos'].tolist(),
                    'bayonet': vt.loc[st_indices,'KKBH'].tolist(),
                    'time': vt.loc[st_indices,'GCSJ'].tolist(),
                    'start': vt.loc[start,'GCSJ'],
                    'stop': vt.loc[end,'GCSJ'],
                    'travel': duration
                })
            st_indices = [idx]
    return segments

# --- 并行执行 ---
start_time = time.time()
all_segments = Parallel(n_jobs=-1)(delayed(process_vehicle)(v) for v in vehicle_list)
all_segments = [seg for sublist in all_segments for seg in sublist]
print(f"Total segments: {len(all_segments)} in {time.time()-start_time:.1f}s")

# --- 结果保存 (原格式) ---
path_pos = [seg['pos'] for seg in all_segments]
path_bayonet = [seg['bayonet'] for seg in all_segments]
path_time = [seg['time'] for seg in all_segments]
path_start_time = [seg['start'] for seg in all_segments]
path_stop_time = [seg['stop'] for seg in all_segments]
path_travel = [seg['travel'] for seg in all_segments]
path_cltmbh = [seg['CLTMBH'] for seg in all_segments]

# 定期保存
np.save(save_path + '/path_pos.npy', path_pos)
np.save(save_path + '/path_bayonet.npy', path_bayonet)
np.save(save_path + '/path_time.npy', path_time)
np.save(save_path + '/path_start_time.npy', path_start_time)
np.save(save_path + '/path_stop_time.npy', path_stop_time)
np.save(save_path + '/path_travel.npy', path_travel)
np.save(save_path + '/path_cltmbh.npy', path_cltmbh)
print("Saved paths in original .npy format.")
